In [33]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import ast
import re

In [3]:
df_path = "data/newsela/articles.csv"

In [2]:
test_urls = ["https://newsela.com/read/elem-sparrow-song/id/44677",
"https://newsela.com/read/gorilla-poop-treasure-hunt/id/44303",
"https://newsela.com/read/jurassic-park-generation/id/44270",
"https://newsela.com/read/school-fights-for-racial-equity/id/44370",
"https://newsela.com/read/can-algorithms-be-art/id/43488",
"https://newsela.com/read/decolonize-student-reading/id/41535",
"https://newsela.com/read/conservative-students-difficult-college-decisions/id/42964",
"https://newsela.com/read/DIY-tech-helps-marine-scientists/id/42157",
"https://newsela.com/read/yanny-laurel-explained/id/43684",
"https://newsela.com/read/elem-mobile-libraries-around-the-world/id/42697"]

# Import Data

In [13]:
base_url = 'https://newsela.com'
url = base_url + '/articles/#/rule/latest'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)


In [41]:
def api_request_newsela(url):
    regex_rm_minus = re.compile(r"(--)*")
    r = requests.get(url)
    text_title = url.split("/")[-1]
    json = r.json()
    level_articles = json["articles"]
    articles = []
    for article in level_articles:
        text = article["teaser"] + "\n" + article["text"]
        text = regex_rm_minus.sub("", text)
        score = article["lexile_level"]
        articles += [(text_title,score,text)]
    return articles

In [19]:
def import_newsela_article_levels(url):
    curr_id = int(url.split("/")[-1]
    url_levels = range(curr_id-100,curr_id+100)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    filters = soup.select("div.styles__dropDownMenu___2d88z:last-child")

[]

In [42]:
api_url = "https://newsela.com/api/v2/articleheader/"
url = "https://newsela.com/read/yanny-laurel-explained/id/43684"
url = api_url + url.split("/")[-3]
api_request_newsela(url)

[('yanny-laurel-explained',
  590,
  'The ear hears what it expects\nPeople have been listening to a recording online. They don\'t all hear the same thing even though it is the same clip. Some people hear the word "laurel." Others hear "yanny."\xa0\n\nIf you have been online lately, you might have heard it. Different people hear it differently. Some people thought it sounded like both words.\xa0\n\nWhat odd trick of the body is going on? What is making people hear two different words from the same bit of sound?\n\nExperts say differences come from the way\xa0we hear\xa0sound waves.\xa0The sounds we expect to hear are important, too.\xa0\n\nBrad Story works for the University of Arizona. He studied the sound recording. Story thinks it was recorded as "laurel." It can be confusing, though, he says. There is a group\xa0of sound waves made by the shape of our throat and mouth, he says.\n\nSimilar Sound Patterns\n\n\nStory recorded his own voice saying both words. He found similar sound pat

In [19]:
page.text

'\n\n<!DOCTYPE html>\n<!--\n  _   _                        _         ____   ___\n | \\ | |                      | |       |___ \\ / _ \\\n |  \\| | _____      _____  ___| | __ _    __) | | | |\n | . ` |/ _ \\ \\ /\\ / / __|/ _ \\ |/ _` |  |__ <| | | |\n | |\\  |  __/\\ V  V /\\__ \\  __/ | (_| |  ___) | |_| |\n |_| \\_|\\___| \\_/\\_/ |___/\\___|_|\\__,_| |____(_)___/\n\n-->\n<html lang="en" class="chrome safari webkit">\n<head>\n  <title>\n    Newsela | Do-it-yourself tech is helping scientists keep up with marine life\n  </title>\n\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UQQCVlRSGwIBUVRQAAgG"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r

In [134]:
def import_britishcouncil_articles():
    base_url = 'https://newsela.com'
    url = base_url + '/articles/#/rule/latest'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    filters = soup.select("div.views-row a")
    
    articles = []
    art_set = set()
    print("Select Articles...")
    for article in filters:
        article_href = article.get_attribute_list("href")[0]
        article_name = article_href.split("/")[-1][:-2]
        article_level = article_href[-1]
        if "1" == article_level and article_name not in art_set: 
            articles += [(article_href[:-2],article_name)]
            art_set.update([article_name])
            
    texts = []
    print("Get Texts...")
    for article in tqdm(articles):
        text_levels = []
        for level in ["-1","-2","-3"]:
            article_page = requests.get(base_url + article[0] + level, headers=headers)
            article_soup = BeautifulSoup(article_page.content, 'html.parser')
            text_parts = [text_part.get_text() for text_part in article_soup.select("div.field-name-body p")]
            text_levels += [text_parts]
        texts += [(article[1],*text_levels)]
        
    return pd.DataFrame(data=texts,columns=["article_name","raw1","raw2","raw3"])

In [165]:
df = import_britishcouncil_articles()

  0%|          | 0/22 [00:00<?, ?it/s]

Select Articles...
Get Texts...


100%|██████████| 22/22 [00:10<00:00,  2.15it/s]


In [166]:
df.head()

,article_name,raw1,raw2,raw3
0,walk-forest-level,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...
1,amazing-adventurers-level,[Do you ever dream about climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...
2,animals-city-level,"[One night in December 2011, a bear came into ...","[Recently, there have been many reports in new...","[Small animals like birds, squirrels, mice and..."
3,bully-level,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...
4,cheat-level,[Mo was waiting outside her class. She was fee...,[Mo was waiting in the corridor outside her cl...,[Mo was waiting in the corridor with her class...


In [183]:
def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

In [167]:
save_df(df_path,df)

# Prepare Data

In [140]:
def load_df(path,list_cols):
    df = pd.read_csv(path,sep="|")
    df[list_cols] = df[list_cols].applymap(
    lambda x: ast.literal_eval(x))
    return df

In [175]:
df = load_df(df_path,["raw1","raw2","raw3"])
df.head()

,article_name,raw1,raw2,raw3
0,walk-forest-level,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...,[Going through the forest is my favourite part...
1,amazing-adventurers-level,[Do you ever dream about climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...,[Have you ever dreamt of climbing Mount Everes...
2,animals-city-level,"[One night in December 2011, a bear came into ...","[Recently, there have been many reports in new...","[Small animals like birds, squirrels, mice and..."
3,bully-level,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...,[Kay got another message as she was leaving fo...
4,cheat-level,[Mo was waiting outside her class. She was fee...,[Mo was waiting in the corridor outside her cl...,[Mo was waiting in the corridor with her class...


In [179]:
replace_strings = ["\xa0\n\t","\n\t","\xa0"]

In [180]:
def clean_data(replace_strings,raw_text):
    for string in replace_strings:
        raw_text = [part.replace(string," ") for part in raw_text]
    return raw_text

In [182]:
df[["cleaned1","cleaned2","cleaned3"]] = df[["raw1","raw2","raw3"]].applymap(lambda x: clean_data(replace_strings,x))

In [185]:
df_cleaned_path = "data/british_council/cleaned_articles.csv"
save_df(df_cleaned_path,df[["article_name","cleaned1","cleaned2","cleaned3"]])